# Import CMIP6 Climate data, aggregate monthly averages and interpolate to different grid

In [ ]:
import numpy as np
from datetime import datetime
import pandas as pd
import xarray as xr
import flox
import cftime

In [ ]:
import glob
import os

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [ ]:
def downsample_sums(filename, var, var_name):

    dataset = xr.open_dataset(filename, decode_times = True)

    dataset = dataset.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))

    data_interp = dataset.interp(lon = lon_arr, lat = lat_arr)
    
    # faster to do this on the xarray
    data_interp['year'] = data_interp['time'].dt.strftime('%Y')
    data_interp['month'] = data_interp['time'].dt.strftime('%B')
    data_interp['day'] = data_interp['time'].dt.strftime('%d')
    #data_interp['time'] = data_interp['time'].dt.strftime('%r')
    
    df = data_interp.to_dataframe()
    df = df.reset_index()

    #df = df[(df['lat'].isin(state_coords.lat_coord2)) & (df['lon'].isin(state_coords.lon_coord2))]

    df_means = df.groupby(['lat','lon','month','year']).agg({var:['sum']})
    df_means.columns = [var_name]
    df_means = df_means.reset_index()

    
    return(df_means)

In [ ]:
def downsample_means(filename, var, var_name):

    dataset = xr.open_dataset(filename, decode_times = True)

    dataset = dataset.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))
    data_interp = dataset.interp(lon = lon_arr, lat = lat_arr)

    # faster to do this on the xarray
    data_interp['year'] = data_interp['time'].dt.strftime('%Y')
    data_interp['month'] = data_interp['time'].dt.strftime('%B')
    data_interp['day'] = data_interp['time'].dt.strftime('%d')
    #data_interp['time'] = data_interp['time'].dt.strftime('%r')
    
    df = data_interp.to_dataframe()
    df = df.reset_index()

    #df = df[(df['lat'].isin(state_coords.lat_coord2)) & (df['lon'].isin(state_coords.lon_coord2))]

    df_means = df.groupby(['lat','lon','month','year']).agg({var:['mean']})
    df_means.columns = [var_name]
    df_means = df_means.reset_index()

    
    return(df_means)

In [ ]:
def get_gdd(tminf, tmaxf):

    dataset_tmin = xr.open_dataset(tminf, decode_times = True)
    dataset_tmax = xr.open_dataset(tmaxf, decode_times = True)

    dataset_tmin = dataset_tmin.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))
    dataset_tmax = dataset_tmax.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))

    tmin_interp = dataset_tmin.interp(lon = lon_arr, lat = lat_arr)
    tmax_interp = dataset_tmax.interp(lon = lon_arr, lat = lat_arr)

    tmin_interp = tmin_interp.merge(tmax_interp)

    tmin_interp['year'] = tmin_interp['time'].dt.strftime('%Y')

    df = tmin_interp.to_dataframe()
    df = df.reset_index()

    df['tasmax'] = df['tasmax'] - 273.15
    df['tasmin'] = df['tasmin'] - 273.15

    df['gdd'] = (df['tasmax'] + df['tasmin'])/2 - 5.6
    df['gdd'] = np.where(df['gdd'] < 0, 0, df['gdd'])

    df['gdd_sum'] = df.groupby(['lat','lon']).cumsum()['gdd']

    #df2 = df[df['gdd_sum'] < 1000]

    return(df)

In [ ]:
def get_season_gdd(tminf, tmaxf):

    dataset_tmin = xr.open_dataset(tminf, decode_times = True)
    dataset_tmax = xr.open_dataset(tmaxf, decode_times = True)

    dataset_tmin = dataset_tmin.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))
    dataset_tmax = dataset_tmax.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))

    tmin_interp = dataset_tmin.interp(lon = lon_arr, lat = lat_arr)
    tmax_interp = dataset_tmax.interp(lon = lon_arr, lat = lat_arr)

    tmin_interp = tmin_interp.merge(tmax_interp)

    tmin_interp['year'] = tmin_interp['time'].dt.strftime('%Y')

    #tmin_interp['time'] = tmin_interp.indexes['time'].to_datetimeindex()

    df = tmin_interp.to_dataframe()
    df = df.reset_index()

    df['tasmax'] = df['tasmax'] - 273.15
    df['tasmin'] = df['tasmin'] - 273.15

    df['gdd'] = (df['tasmax'] + df['tasmin'])/2 - 5.6
    df['gdd'] = np.where(df['gdd'] < 0, 0, df['gdd'])

    df['gdd_sum'] = df.groupby(['lat','lon']).cumsum()['gdd']

    df['julian'] = pd.DatetimeIndex(df['time']).dayofyear
    hatch_pred = df[df.gdd_sum>= 300].groupby(['lat','lon','year']).min('julian')
    hatch_pred['julian'] = hatch_pred['julian'] + 69

    hatch_pred_small = hatch_pred.reset_index()[['lat','lon','year','julian']]

    gdd_before = df[df.gdd_sum< 300].groupby(['lat','lon','year']).max('julian').reset_index()[['lat','lon','year','gdd_sum']].rename(columns = {'gdd_sum' : 'gdd_subtract'})

    season_gdds = pd.merge(hatch_pred_small,df, how = 'left')

    season_gdds = pd.merge(season_gdds,gdd_before,how = 'left')

    season_gdds['gdd_season'] = season_gdds['gdd_sum'] - season_gdds['gdd_subtract']
    season_gdds = season_gdds[['lat','lon','year','julian','gdd_season']]

    return(season_gdds)

In [ ]:
def get_season_gdd_360(tminf, tmaxf):

    dataset_tmin = xr.open_dataset(tminf, decode_times = True)
    dataset_tmax = xr.open_dataset(tmaxf, decode_times = True)

    dataset_tmin = dataset_tmin.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))
    dataset_tmax = dataset_tmax.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))

    tmin_interp = dataset_tmin.interp(lon = lon_arr, lat = lat_arr)
    tmax_interp = dataset_tmax.interp(lon = lon_arr, lat = lat_arr)

    tmin_interp = tmin_interp.merge(tmax_interp)

    tmin_interp['year'] = tmin_interp['time'].dt.strftime('%Y')
    tmin_interp['month'] = tmin_interp['time'].dt.strftime('%m')
    tmin_interp['day'] = tmin_interp['time'].dt.strftime('%d')

    df = tmin_interp.to_dataframe()
    df = df.reset_index()

    df.month =  pd.to_numeric(df.month, errors='coerce')
    df.day = pd.to_numeric(df.day, errors='coerce')

    df['julian'] = (df.month-1)*30 + df.day

    df['tasmax'] = df['tasmax'] - 273.15
    df['tasmin'] = df['tasmin'] - 273.15

    df['gdd'] = (df['tasmax'] + df['tasmin'])/2 - 5.6
    df['gdd'] = np.where(df['gdd'] < 0, 0, df['gdd'])

    df['gdd_sum'] = df.groupby(['lat','lon']).cumsum()['gdd']

    hatch_pred = df[df.gdd_sum>= 300].groupby(['lat','lon','year']).min('julian')
    hatch_pred['julian'] = hatch_pred['julian'] + 69

    hatch_pred_small = hatch_pred.reset_index()[['lat','lon','year','julian']]

    gdd_before = df[df.gdd_sum< 300].groupby(['lat','lon','year']).max('julian').reset_index()[['lat','lon','year','gdd_sum']].rename(columns = {'gdd_sum' : 'gdd_subtract'})

    season_gdds = pd.merge(hatch_pred_small,df, how = 'left')

    season_gdds = pd.merge(season_gdds,gdd_before,how = 'left')

    season_gdds['gdd_season'] = season_gdds['gdd_sum'] - season_gdds['gdd_subtract']
    season_gdds = season_gdds[['lat','lon','year','julian','gdd_season']]

    return(season_gdds)

In [ ]:
lat_min = 32
lat_max = 54
lon_min = 360 - 128
lon_max = 360 - 100

lat_arr = np.arange(lat_min,lat_max,0.25)
lon_arr = np.arange(lon_min,lon_max,0.25)


In [ ]:
climate_models = ["ACCESS-ESM1-5","CanESM5-p1","EC-Earth3-Veg-LR","CNRM-ESM2-f2","GFDL-ESM4","HadGEM3-GC31-MM","INM-CM5-0","KACE-1-0-G","MIROC-ES2L-f2","NorESM2-MM"]
year_sets1 = ["2030","2050","2070","2090","2041"]
year_sets2 = ["2040","2060","2080","2100","2089"]

In [ ]:
clim_id = 2
yr_id = 4

In [ ]:
model_dir = "/Volumes/My Book/Climate/CMIP6/" + climate_models[clim_id] + "_ssp585_" + year_sets1[yr_id] + "/"
mod_name = climate_models[clim_id]
year1 = year_sets1[yr_id]
year2 = year_sets2[yr_id]

print(model_dir)

In [ ]:
filenames = listdir_nohidden(model_dir + "tas/")

tas_df = pd.DataFrame()

for file in filenames:

    tas_df = tas_df.append(downsample_means(file, 'tas','tas_mean'))

tas_df.to_csv(model_dir + "downsampled/" + "means_tas_" + year1 + "-" + year2 + "_" + mod_name + ".csv")

In [ ]:
filenames = listdir_nohidden(model_dir + "tasmax/")

#dataset = xr.open_dataset(filenames[1], decode_times = True)

tmax_df = pd.DataFrame()

for file in filenames:

    tmax_df = tmax_df.append(downsample_means(file, 'tasmax','tmax_mean'))

tmax_df.to_csv(model_dir + "downsampled/" + "means_tmax_" + year1 + "-" + year2 + "_" + mod_name + ".csv")

In [ ]:
filenames = listdir_nohidden(model_dir + "tasmin/")

tmin_df = pd.DataFrame()

for file in filenames:

    tmin_df = tmin_df.append(downsample_means(file, 'tasmin','tmin_mean'))

tmin_df.to_csv(model_dir + "downsampled/" + "means_tmin_" + year1 + "-" + year2 + "_" + mod_name + ".csv")

In [ ]:
filenames = listdir_nohidden(model_dir + "hum/")

hurs_df = pd.DataFrame()

for file in filenames:

    hurs_df = hurs_df.append(downsample_means(file, 'hurs','hurs_mean'))

hurs_df.to_csv(model_dir + "downsampled/" + "means_hurs_" + year1 + "-" + year2 + "_" + mod_name + ".csv")

In [ ]:
filenames = listdir_nohidden(model_dir + "pr/")

pr_df = pd.DataFrame()

for file in filenames:

    pr_df = pr_df.append(downsample_sums(file, 'pr','sum_pr'))

pr_df.to_csv(model_dir + "downsampled/" + "means_pr_" + year1 + "-" +year2 + "_" + mod_name + ".csv")

In [ ]:
file_tmin = listdir_nohidden(model_dir + "tasmin/")
file_tmax = listdir_nohidden(model_dir + "tasmax/")

print(file_tmin[0])
print(file_tmax[0])

file_len = len(file_tmin[0])

if year1 == '2041':
    years = np.concatenate((np.arange(2041,2050),np.arange(2061,2070),np.arange(2081,2090)))    
else:
    years =  np.arange(int(year1),int(year2)+1,1)
gdd_df = pd.DataFrame()

for yr in years:

    tmin_file = file_tmin[0][0:(file_len -7)] + str(yr) + '.nc'
    tmax_file = file_tmax[0][0:(file_len -7)] + str(yr) + '.nc'

    if mod_name in ("HadGEM3-GC31-MM","KACE-1-0-G"):
        gdd_df = gdd_df.append(get_season_gdd_360(tmin_file,tmax_file))
    else:
        gdd_df = gdd_df.append(get_season_gdd(tmin_file,tmax_file))


gdd_df.to_csv(model_dir + "downsampled/" + "ALL_season_gdd_" + year1 + "-" + year2 + "_" + mod_name + ".csv")

In [ ]:
mod_name

In [ ]:
def slice_to_df(filename):

    dataset = xr.open_dataset(filename, decode_times = True)

    dataset = dataset.sel(lon = slice(lon_min,lon_max), lat = slice(lat_min,lat_max))

    data_interp = dataset.interp(lon = lon_arr, lat = lat_arr)

    # faster to do this on the xarray
    data_interp['year'] = data_interp['time'].dt.strftime('%Y')
    data_interp['month'] = data_interp['time'].dt.strftime('%B')
    data_interp['day'] = data_interp['time'].dt.strftime('%d')
    data_interp['time'] = data_interp['time'].dt.strftime('%r')
    
    df = data_interp.to_dataframe()
    df = df.reset_index()

    return(df)

In [ ]:
var = 'tasmin'

filenames = listdir_nohidden(model_dir + var + "/")

#print(filenames)

df = pd.DataFrame()

for file in filenames:

    df = df.append(slice_to_df(file))
    print(f"finished file: {file}")

df.to_csv(model_dir + "downsampled_int/" + "ALL_" + var + "_" + year1 + "-" + year2 + "_GFDL-ESM4.csv")

print("Fini")